<a href="https://colab.research.google.com/github/shivani1912/ProductReviews_ETL/blob/master/ProductReviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Objective: Upload previously scraped amazon reviews into MongoDB and analyze review scores using MapReduce

- Create a MongoDB database called "amazon".

In [ ]:
import json
from pymongo import MongoClient

#connecting to the local host
client = MongoClient('localhost', 27017)

#create a database named amazon
db = client['amazon']
#create a collection named reviews
collection_reviews = db['reviews']



- Read "reviews_electronics.16.json" and upload each review as a separate document to the collection "reviews" in the database "amazon".

In [ ]:
import json
#import data from the mongo dump json to data
data = [json.loads(line) for line in open('reviews_electronics.16.json', 'r')]

#use insert_many() for inserting many documents
collection_reviews.insert_many(data)
#For QC
for i in collection_reviews.find({}).limit(10):
    print(i)

#client.close()

{'_id': ObjectId('5e4d016313b3adc6a3ec0eb1'), 'reviewerID': 'AKM1MP6P0OYPR', 'asin': '0132793040', 'reviewerName': 'Vicki Gibson "momo4"', 'helpful': [1, 1], 'reviewText': 'Corey Barker does a great job of explaining Blend Modes in this DVD. All of the Kelby training videos are great but pricey to buy individually. If you really want bang for your buck just subscribe to Kelby Training online.', 'overall': 5.0, 'summary': 'Very thorough', 'unixReviewTime': 1365811200, 'reviewTime': '04 13, 2013'}
{'_id': ObjectId('5e4d016313b3adc6a3ec0eb2'), 'reviewerID': 'A2X8VX4DPMQFQQ', 'asin': 'B00E4KP4W6', 'reviewerName': 'lily68', 'helpful': [1, 1], 'reviewText': "I can't believe I waited to long to switch to a glass screen protector.  I love this.  It feels and looks like there is no protector on.  It does show fingerprints, which I think is inevitable unless you use a matte finish screen protector, but they wipe right away. I would definitely recommend this! Easier to apply than the films too!",

#### Part 1
- Use MongoDB's map reduce function to build a new collection "avg_scores" that averages review scores by product ("asin"). Print the first 100 entries of "avg_scores" to screen.


In [ ]:
from bson.code import Code

#Create the map function to obtain overall scores
map = Code('''function(){emit(this.asin,this.overall);}''')
#Reduce it to asin level and compute average
reduce = Code('''function(key,values){return Array.avg(values)}''')

#final map reduce to get desireed results
result1 = db.reviews.map_reduce(map, reduce, "avg_scores")

#print first 100 rows
for doc in result1.find().limit(100):
    print(doc)

{'_id': '0132793040', 'value': 5.0}
{'_id': 'B00E4KP4W6', 'value': 4.545454545454546}
{'_id': 'B00E4KP8VI', 'value': 5.0}
{'_id': 'B00E4KPMC8', 'value': 2.0}
{'_id': 'B00E4KQ5C4', 'value': 5.0}
{'_id': 'B00E4KQ9GG', 'value': 3.2857142857142856}
{'_id': 'B00E4KQ9K2', 'value': 5.0}
{'_id': 'B00E4KQD4E', 'value': 4.0}
{'_id': 'B00E4KZBX8', 'value': 4.0}
{'_id': 'B00E4KZDJ0', 'value': 5.0}
{'_id': 'B00E4L35DA', 'value': 4.0}
{'_id': 'B00E4L3N9Q', 'value': 4.0}
{'_id': 'B00E4L48EA', 'value': 5.0}
{'_id': 'B00E4L7FLI', 'value': 1.0}
{'_id': 'B00E4L7TS2', 'value': 4.0}
{'_id': 'B00E4LAL82', 'value': 3.0}
{'_id': 'B00E4LBZZK', 'value': 5.0}
{'_id': 'B00E4LF2Z4', 'value': 4.333333333333333}
{'_id': 'B00E4LFP0G', 'value': 4.444444444444445}
{'_id': 'B00E4LFWWW', 'value': 4.4}
{'_id': 'B00E4LGTVU', 'value': 4.173537012873982}
{'_id': 'B00E4LGTXS', 'value': 3.923076923076923}
{'_id': 'B00E4LGVYA', 'value': 3.272727272727273}
{'_id': 'B00E4LGWLW', 'value': 3.5}
{'_id': 'B00E4LGXL6', 'value': 5.0}
{

In [ ]:
import os
os.getcwd()

'C:\\Users\\shali\\OneDrive - University of California, Davis\\Documents\\3 WQ Data Design\\Assignments'


- Use MongoDB's map reduce function to build a new collection "weighted_avg_scores" that averages review scores by product<br> ("asin"), weighted by the number of helpful votes (The base weight is 1 and for every additional helpful vote add 1 to weight.<br> e.g. a "[3, 5]" value on "helpful" column should use 3 + 1 = 4 as weight,<br> 3 being the additional votes and 1 being the base weight).<br> Print the first 100 entries of "weighted_avg_scores" to screen.

In [ ]:
from bson.code import Code
#Map function to get compute weight and multiply it with review
map = Code('''function(){
                            var val =this.helpful[0] + 1;
                            var wt_value ={ wt: val,
                                         wt_value: this.overall * val,
                                        };
                            emit( this.asin, wt_value );
                            }''')
#compute weighted average at asin level
reduce = Code('''function(key, value) {
                     reducedVal = { wt: 0, wt_value: 0 };

                     for (var idx = 0; idx < value.length; idx++) {
                         reducedVal.wt += value[idx].wt;
                         reducedVal.wt_value += value[idx].wt_value;
                     }

                     return reducedVal;
                  };''')

finalize= Code('''function (key, reducedVal) {

                       reducedVal.wt_avg = reducedVal.wt_value/reducedVal.wt;

                       return reducedVal;

                    };''')

#final map reduce function
result2 = db.reviews.map_reduce(map, reduce, "weighted_avg_scores",finalize=finalize)

#print first 100 rows
for doc in result2.find().limit(100):
    print(doc)

{'_id': '0132793040', 'value': {'wt': 8.0, 'wt_value': 40.0, 'wt_avg': 5.0}}
{'_id': 'B00E4KP4W6', 'value': {'wt': 76.0, 'wt_value': 356.0, 'wt_avg': 4.684210526315789}}
{'_id': 'B00E4KP8VI', 'value': {'wt': 4.0, 'wt_value': 20.0, 'wt_avg': 5.0}}
{'_id': 'B00E4KPMC8', 'value': {'wt': 24.0, 'wt_value': 48.0, 'wt_avg': 2.0}}
{'_id': 'B00E4KQ5C4', 'value': {'wt': 8.0, 'wt_value': 40.0, 'wt_avg': 5.0}}
{'_id': 'B00E4KQ9GG', 'value': {'wt': 64.0, 'wt_value': 236.0, 'wt_avg': 3.6875}}
{'_id': 'B00E4KQ9K2', 'value': {'wt': 52.0, 'wt_value': 260.0, 'wt_avg': 5.0}}
{'_id': 'B00E4KQD4E', 'value': {'wt': 4.0, 'wt_value': 16.0, 'wt_avg': 4.0}}
{'_id': 'B00E4KZBX8', 'value': {'wt': 8.0, 'wt_value': 32.0, 'wt_avg': 4.0}}
{'_id': 'B00E4KZDJ0', 'value': {'wt': 4.0, 'wt_value': 20.0, 'wt_avg': 5.0}}
{'_id': 'B00E4L35DA', 'value': {'wt': 48.0, 'wt_value': 144.0, 'wt_avg': 3.0}}
{'_id': 'B00E4L3N9Q', 'value': {'wt': 4.0, 'wt_value': 16.0, 'wt_avg': 4.0}}
{'_id': 'B00E4L48EA', 'value': {'wt': 4.0, 'wt_val

In [ ]:
client.close()